In [1]:
import ipywidgets as widgets

In [2]:
train_data = catalog.load("train_data")
test_data = catalog.load("eval_data")

2022-02-28 13:05:13,953 - kedro.io.data_catalog - INFO - Loading data from `train_data` (CSVDataSet)...
2022-02-28 13:05:14,004 - kedro.io.data_catalog - INFO - Loading data from `eval_data` (CSVDataSet)...


In [3]:
train_data

,serie_id,date,sales,dcoilwtico,group
0,"(10, 'BOOKS')",2013-01-01,0.0,0.00,0
1,"(10, 'BOOKS')",2013-01-02,0.0,93.14,0
2,"(10, 'BOOKS')",2013-01-03,0.0,92.97,0
3,"(10, 'BOOKS')",2013-01-04,0.0,93.12,0
4,"(10, 'BOOKS')",2013-01-07,0.0,93.20,0
...,...,...,...,...,...
59495,"(9, 'BOOKS')",2017-07-18,0.0,46.40,0
59496,"(9, 'BOOKS')",2017-07-19,0.0,47.10,0
59497,"(9, 'BOOKS')",2017-07-20,0.0,46.73,0
59498,"(9, 'BOOKS')",2017-07-21,0.0,45.78,0


In [4]:
from ipywidgets import *
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
import warnings 
warnings.filterwarnings("ignore")
sns.set_style("darkgrid")

In [5]:
sns.__version__

'0.11.2'

In [6]:
def fit_and_pred(model, model_name, train, y_true, train_data, test_data, X, X_test):
    model.fit(y=train, X=X)
    y_pred = model.predict(y_true.shape[0], X=X_test)
    y_pred[y_pred < 0] = 0
    
    error = np.sqrt(mean_squared_log_error(y_true + 1, y_pred + 1))
    pred_data = test_data.copy()
    pred_data["sales"] = y_pred.values
    
    pred_data["Validation"] = model_name
    serie_data = pd.concat((train_data, pred_data), ignore_index=True)
    return serie_data, error

In [9]:
from time_series_kedro.extras.models import *
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error, mean_squared_log_error

def plot_serie(
    serie_id=(5, "LADIESWEAR"), 
    start=2017,
    seasonal_periods=2,
    seasonal=None,
    trend=None):
    train_serie_data = train_data[train_data.serie_id == serie_id]
    train_serie_data["Validation"] = "train"
    test_serie_data = test_data[test_data.serie_id == serie_id]
    test_serie_data["Validation"] = "test"
    train_serie_data = train_serie_data[(train_serie_data.date.dt.year) >= start]
    serie_data = pd.concat((train_serie_data, test_serie_data), ignore_index=True)
    
    X = train_serie_data[["date", "dcoilwtico"]].set_index("date")
    X_test = test_serie_data[["date", "dcoilwtico"]].set_index("date")
    train = train_serie_data.set_index("date").sales
    y_true = test_serie_data.set_index("date").sales
    
    train = train + 1
    serie_data, error = fit_and_pred(ExponentialSmoothing(seasonal_periods=seasonal_periods, seasonal=seasonal, trend=trend),
                              "es", train, y_true, serie_data, test_serie_data, X=None, X_test=None)
    serie_data, error = fit_and_pred(Prophet(period=seasonal_periods, seasonality=seasonal),
                              "prophet", train, y_true, serie_data, test_serie_data, X=X, X_test=X_test)
    fig = plt.figure(figsize=(18, 10))
    ax = fig.add_subplot(1, 1, 1)
    serie_data = serie_data[(serie_data.date.dt.year) >= 2017]
    ax.set_title(f"Serie group: {serie_data.group.values[0]}, Error: {error:.2f}")
    sns.lineplot(x="date", y="sales", data=serie_data, ax=ax, hue="Validation")
    
   #serie_data.sales.plot(ax=ax)
interact(plot_serie, 
         serie_id=train_data[train_data.group > 0].serie_id.unique(), 
         start=range(2013, 2018, 1),
         seasonal=["additive", "multiplicative", None],
         trend=["additive", None],
         seasonal_periods=(2,30,1));

interactive(children=(Dropdown(description='serie_id', options=("(11, 'LAWN AND GARDEN')", "(19, 'CLEANING')",…

In [8]:
serie_cols = ["store_nbr", "family"]
train_data['serie'] = list(map(str, zip(*[train_data[c] for c in agrup])))

NameError: name 'agrup' is not defined